# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 11:11:09,172 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [2]:
# TWrite a prompt to generate happy path tests for your API.
happy_path_tests_prompt = f"""
You are an expert QA Engineer specializing in pytest and FastAPI testing. Your task is to generate comprehensive "happy path" test functions for a FastAPI application.

**Context - Application Code:**
{app_code}

**Instructions:**
Generate a pytest test file with the following requirements:

1. **Test Structure:**
   - Use pytest best practices with clear, descriptive test function names following the pattern `test_<operation>_<expected_outcome>`
   - Include proper imports: `pytest`, `fastapi.testclient.TestClient`, and import the `app` from `app.main`
   - Create a module-level TestClient instance: `client = TestClient(app)`

2. **Required Happy Path Tests:**
   a) **Test POST /users/** - Create a new user successfully:
      - Send a valid user payload with all required fields (first_name, last_name, email, role_id, start_date)
      - Assert the response status code is 201 (Created)
      - Assert the response contains the created user data with a user_id
      - Assert all input fields match the response fields
      - Use a unique email address (e.g., with timestamp or UUID) to avoid conflicts
   
   b) **Test GET /users/** - List all users:
      - Send a GET request to retrieve all users
      - Assert the response status code is 200 (OK)
      - Assert the response is a list (can be empty or contain users)
      - If users exist, verify each user has the required fields (user_id, first_name, last_name, email, role_id, start_date)
   
   c) **Test GET /users/{{user_id}}** - Get a specific user:
      - First create a user via POST /users/
      - Then retrieve that user by their user_id
      - Assert the response status code is 200 (OK)
      - Assert the retrieved user data matches the created user data

3. **Best Practices to Follow:**
   - Each test should be independent and not rely on specific database state
   - Use descriptive variable names
   - Add docstrings to each test function explaining what it tests
   - Use proper assertions with clear failure messages
   - Clean up any test data if necessary (though fixtures will handle this later)
   - Follow the Arrange-Act-Assert (AAA) pattern in each test

4. **Output Format:**
   - Provide only valid Python code for the test file
   - Do not include markdown code fences or explanatory text outside the code
   - Include a module-level docstring explaining the purpose of the test file
   - Ensure the code is properly formatted and follows PEP 8 style guidelines

Generate the complete, production-ready pytest test file now.
"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py", overwrite = True)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
"""
This module contains "happy path" tests for the Onboarding Task Management API.

It uses the FastAPI TestClient to send requests to the application and verify
the successful creation and retrieval of users. Each test is designed to be
independent and follows the Arrange-Act-Assert pattern.
"""

import datetime
import uuid

from fastapi.testclient import TestClient

# Assuming the application code is in a file named `main.py`.
# If your file is named differently, please adjust the import accordingly.
from main import app

# Create a TestClient instance to interact with the FastAPI application
client = TestClient(app)


def test_create_user_success():
    """
    Tests the successful creation of a new user via the POST /users endpoint.

    It first creates a prerequisite role, then sends a valid user payload.
    It asserts that the response status is 201 (Created) and that the
    returned user data matches the input data.
    """
    # Arrange
 

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [3]:
# TODO: Write a prompt to generate edge case tests.
edge_case_tests_prompt = f"""
You are an expert QA Engineer specializing in pytest and FastAPI testing. Your task is to generate comprehensive edge case tests for a FastAPI application that handles error scenarios gracefully.

**Context - Application Code:**
{app_code}

**Instructions:**
Generate a pytest test file with edge case tests that verify proper error handling:

1. **Test Structure:**
   - Use pytest best practices with clear, descriptive test function names following the pattern `test_<operation>_<error_scenario>`
   - Include proper imports: `pytest`, `fastapi.testclient.TestClient`, and import the `app` from `app.main`
   - Create a module-level TestClient instance: `client = TestClient(app)`
   - Add a module-level docstring explaining that this file contains edge case and error handling tests

2. **Required Edge Case Tests:**

   a) **Test POST /users/ with duplicate email** - Verify duplicate email handling:
      - First, create a user with a specific email address
      - Then, attempt to create another user with the same email
      - Assert the second request returns status code 400 (Bad Request)
      - Assert the error response contains a meaningful error message (e.g., "Email already exists")
      - Include a docstring explaining this tests duplicate email constraint

   b) **Test GET /users/{{user_id}} with non-existent ID** - Verify 404 handling:
      - Send a GET request for a user_id that doesn't exist (e.g., 999999)
      - Assert the response status code is 404 (Not Found)
      - Assert the error response contains a meaningful error message (e.g., "User not found")
      - Include a docstring explaining this tests missing resource handling

3. **Best Practices to Follow:**
   - Each test should be independent and properly isolated
   - Use descriptive variable names that clearly indicate the test scenario
   - Add comprehensive docstrings to each test function explaining what error condition it tests
   - Use pytest.raises() where appropriate for exception testing (though FastAPI returns HTTP errors)
   - Include clear assertion messages that help diagnose failures
   - Follow the Arrange-Act-Assert (AAA) pattern
   - For tests that create data first, ensure they use unique identifiers to avoid conflicts

4. **Output Format:**
   - Provide only valid Python code for the test file
   - Do not include markdown code fences or explanatory text outside the code
   - Include a comprehensive module-level docstring
   - Ensure the code is properly formatted and follows PEP 8 style guidelines
   - Group related tests logically with comments if helpful

Generate the complete, production-ready pytest edge case test file now.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
"""
Module for edge case and error handling tests for the Onboarding Task Management API.

This file contains pytest tests that specifically target scenarios where the API
is expected to return an error response, such as creating duplicate resources,
requesting non-existent items, or providing invalid foreign keys.
"""
import datetime

import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool

# Assuming the application code is in a file named `main.py` in the parent directory
# or a location accessible via PYTHONPATH.
from main import app, get_db, Base

# --- Test Database Setup ---
# Use an in-memory SQLite database for isolated testing
SQLALCHEMY_DATABASE_URL = "sqlite:///:memory:"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    connect_args={"check_same_thread": False},
    poolclass=StaticPool,
)
TestingSessionLocal = sessio

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [ ]:
# TODO: Write a prompt to generate the pytest fixture for an isolated test database.
db_fixture_prompt = f"""
You are an expert QA Engineer specializing in pytest fixtures and FastAPI testing with database isolation. Your task is to generate a professional-grade pytest fixture for database testing.

**Context - Application Code:**
{app_code}

**Instructions:**
Generate a `conftest.py` file for pytest with the following requirements:

1. **File Purpose:**
   - This file should contain pytest fixtures that will be automatically discovered and used by all test files
   - Include a comprehensive module-level docstring explaining that this provides shared fixtures for database isolation
   - The fixture must create an isolated, in-memory SQLite database for testing

2. **Required Imports:**
   - Import necessary modules: `pytest`, `sqlalchemy` components, `fastapi.testclient.TestClient`
   - Import the FastAPI `app`, database models (`Base`), and the `get_db` dependency from `app.main`
   - Import any other necessary SQLAlchemy components for database creation

3. **Database Fixture Requirements:**
   
   a) **Create a `test_db` fixture** with the following characteristics:
      - Use `@pytest.fixture(scope="function")` to create a fresh database for each test function
      - Create an in-memory SQLite database using `sqlite:///:memory:`
      - Use SQLAlchemy to create a new engine with proper configuration
      - Create all database tables using `Base.metadata.create_all(bind=engine)`
      - Create a SessionLocal for the test database
      - Yield a database session to the test
      - After the test completes, properly close the session and drop all tables using `Base.metadata.drop_all(bind=engine)`
      - Include error handling to ensure cleanup happens even if tests fail
   
   b) **Create a `client` fixture** that:
      - Depends on the `test_db` fixture (use `test_db` as a parameter)
      - Overrides the FastAPI app's `get_db` dependency to use the test database instead of the production database
      - Uses `app.dependency_overrides[get_db]` to replace the dependency
      - Returns a `TestClient(app)` instance for making API requests
      - After the test completes, clears the dependency override with `app.dependency_overrides.clear()`
      - Properly handles setup and teardown

4. **Best Practices to Follow:**
   - Use proper fixture scopes (`function` scope ensures test isolation)
   - Include try-finally blocks to ensure cleanup happens
   - Add comprehensive docstrings to each fixture explaining its purpose and usage
   - Ensure the fixture properly yields control to tests
   - Make sure database sessions are properly closed
   - Guarantee that each test starts with a clean, empty database
   - Use the `yield` keyword correctly for setup/teardown patterns

5. **Critical Implementation Details:**
   - The `test_db` fixture must create a function that returns the test database session
   - This function should be passed to `app.dependency_overrides[get_db]` in the client fixture
   - Ensure compatibility with the existing `get_db` dependency pattern in the main application
   - The fixture should handle SQLAlchemy session management properly
   - Include proper error handling for database operations

6. **Output Format:**
   - Provide only valid Python code for the conftest.py file
   - Do not include markdown code fences or explanatory text outside the code
   - Ensure the code is properly formatted and follows PEP 8 style guidelines
   - Include clear comments where helpful for understanding complex fixture behavior

Generate the complete, production-ready conftest.py file now.
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py", overwrite=True)
else:
    print("Skipping fixture generation because app context is missing.")

# TODO: Write a prompt to refactor the happy path tests to use the new fixture.
refactor_tests_prompt = f"""
You are an expert QA Engineer specializing in pytest and FastAPI testing. Your task is to refactor existing happy path tests to use a pytest fixture for database isolation.

**Context - Application Code:**
{app_code}

**Instructions:**
Generate a refactored pytest test file with the following requirements:

1. **File Purpose:**
   - This file contains the same happy path tests from Challenge 1, but refactored to use the database fixture from conftest.py
   - Include a module-level docstring explaining that these are happy path tests using isolated database fixtures
   - Tests should NOT create their own TestClient - they will use the fixture instead

2. **Test Structure:**
   - Use pytest best practices with clear, descriptive test function names following the pattern `test_<operation>_<expected_outcome>`
   - Include proper imports: `pytest`, and import models/schemas if needed for type checking
   - **DO NOT create a module-level TestClient** - instead, each test function should accept a `client` parameter
   - The `client` parameter will automatically receive the fixture from conftest.py

3. **Required Happy Path Tests (Refactored):**

   a) **Test POST /users/** - Create a new user successfully:
      - Function signature: `def test_create_user_success(client):`
      - Accept `client` as a fixture parameter (pytest will inject it automatically)
      - Send a valid user payload with all required fields (first_name, last_name, email, role_id, start_date)
      - Assert the response status code is 201 (Created)
      - Assert the response contains the created user data with a user_id
      - Assert all input fields match the response fields
      - Use a unique email address to ensure test isolation
   
   b) **Test GET /users/** - List all users:
      - Function signature: `def test_list_users_success(client):`
      - Accept `client` as a fixture parameter
      - First, create one or two test users via POST to ensure the database has data
      - Then send a GET request to retrieve all users
      - Assert the response status code is 200 (OK)
      - Assert the response is a list containing the created users
      - Verify each user has the required fields (user_id, first_name, last_name, email, role_id, start_date)
   
   c) **Test GET /users/{{user_id}}** - Get a specific user:
      - Function signature: `def test_get_user_by_id_success(client):`
      - Accept `client` as a fixture parameter
      - First create a user via POST /users/
      - Extract the user_id from the response
      - Then retrieve that user by their user_id
      - Assert the response status code is 200 (OK)
      - Assert the retrieved user data matches the created user data

   d) **Test PUT /users/{{user_id}}** - Replace/update an entire user:
      - Function signature: `def test_update_user_success(client):`
      - Accept `client` as a fixture parameter
      - First create a user
      - Then send a PUT request with completely new data for all fields
      - Assert the response status code is 200 (OK)
      - Assert all fields are updated to the new values

   e) **Test DELETE /users/{{user_id}}** - Delete a user:
      - Function signature: `def test_delete_user_success(client):`
      - Accept `client` as a fixture parameter
      - First create a user
      - Then send a DELETE request for that user_id
      - Assert the response status code is 204 (No Content)
      - Verify the user is actually deleted by trying to GET it (should return 404)

4. **Key Differences from Original Tests:**
   - Each test function MUST accept `client` as a parameter (this is the fixture injection)
   - NO module-level `client = TestClient(app)` - the fixture handles this
   - Tests can assume they start with a completely empty, fresh database
   - Tests don't need to worry about existing data or conflicts (except within the same test)
   - Each test runs in isolation with its own database

5. **Best Practices to Follow:**
   - Each test is independent and starts with a clean database (thanks to the fixture)
   - Use descriptive variable names
   - Add docstrings to each test function explaining what it tests
   - Use proper assertions with clear failure messages
   - Follow the Arrange-Act-Assert (AAA) pattern
   - For tests that need data, create it at the start of the test

6. **Output Format:**
   - Provide only valid Python code for the test file
   - Do not include markdown code fences or explanatory text outside the code
   - Include a module-level docstring explaining the purpose of the test file
   - Ensure the code is properly formatted and follows PEP 8 style guidelines

Generate the complete, production-ready pytest test file with fixture-based tests now.
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", overwrite= True)
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
"""
conftest.py: Shared pytest fixtures for database-isolated testing.

This file provides fixtures for setting up a clean, isolated, in-memory SQLite
database for each test function. This ensures that tests are independent and
do not interfere with each other or the production database.

Fixtures provided:
- `test_db`: Creates and tears down an in-memory SQLite database for each test.
             It yields a SQLAlchemy session object for direct database interaction.
- `client`: Depends on `test_db` to create a FastAPI `TestClient`. It overrides
            the application's `get_db` dependency to use the isolated test
            database session, allowing for full integration testing of API
            endpoints with a clean database state.
"""

import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker

# Import the main application, the declarative base, and the 

: 

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.

## Bonus: View the API in Browser

You can view the FastAPI application's interactive documentation in your browser. The application should already be running from the terminal.

In [ ]:
import webbrowser
import time

# API Base URL
api_url = "http://127.0.0.1:8000"

# FastAPI automatically provides interactive API documentation
# Swagger UI (interactive API docs)
swagger_url = f"{api_url}/docs"

# ReDoc (alternative API docs)
redoc_url = f"{api_url}/redoc"

print("Opening FastAPI application in browser...")
print(f"\nAPI Base URL: {api_url}")
print(f"Swagger UI (Interactive Docs): {swagger_url}")
print(f"ReDoc (Alternative Docs): {redoc_url}")
print("\nNote: Make sure your FastAPI server is running in the terminal!")
print("If not running, execute this command in the terminal:")
print("  cd C:\\Users\\labadmin\\Documents\\AG-AISOFTDEV")
print("  python -m uvicorn app.main:app --reload")

# Open Swagger UI in the default browser
webbrowser.open(swagger_url)

print("\n✓ Browser opened with Swagger UI")
print("\nYou can:")
print("  - View all available endpoints")
print("  - Test the API directly from the browser")
print("  - See request/response schemas")
print("  - Try out POST, GET, PUT, DELETE operations")

### Run the Tests

Now let's run the generated tests to verify they work correctly. We'll run them with pytest and check for any errors.

In [ ]:
import subprocess
import sys

print("=== Running Tests with Pytest ===\n")

# Test files to run
test_targets = [
    ("Happy Path Tests (Simple)", "tests/test_main_simple.py"),
    ("Database Fixture Tests", "tests/test_main_with_fixture.py"),
]

for test_name, test_file in test_targets:
    full_path = os.path.join(project_root, test_file)
    
    if not os.path.exists(full_path):
        print(f"\n⊘ {test_name}")
        print(f"   File not found: {test_file}")
        print("   Skipping...\n")
        continue
    
    print(f"\n{'='*60}")
    print(f"Running: {test_name}")
    print(f"File: {test_file}")
    print('='*60 + "\n")
    
    # Run pytest with verbose output
    cmd = [sys.executable, "-m", "pytest", test_file, "-v", "--tb=short"]
    
    try:
        result = subprocess.run(
            cmd,
            cwd=project_root,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        # Show output
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print("STDERR:", result.stderr)
        
        # Check result
        if result.returncode == 0:
            print(f"\n✓ {test_name} - ALL TESTS PASSED")
        else:
            print(f"\n✗ {test_name} - SOME TESTS FAILED (Exit code: {result.returncode})")
            
    except subprocess.TimeoutExpired:
        print(f"✗ {test_name} - TIMEOUT (tests took too long)")
    except Exception as e:
        print(f"✗ {test_name} - ERROR: {str(e)}")

print("\n" + "="*60)
print("Test verification complete!")
print("="*60)

=== Running Tests with Pytest ===


Running: Happy Path Tests (Simple)
File: tests/test_main_simple.py

============================= test session starts =============================
platform win32 -- Python 3.11.9, pytest-8.4.2, pluggy-1.6.0 -- c:\Users\labadmin\Documents\AG-AISOFTDEV\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\labadmin\Documents\AG-AISOFTDEV
configfile: pytest.ini
plugins: anyio-4.11.0, langsmith-0.4.38, asyncio-1.2.0
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function
collecting ... collected 3 items

tests/test_main_simple.py::test_create_user_success PASSED               [ 33%]
tests/test_main_simple.py::test_get_users_list_success PASSED            [ 66%]
tests/test_main_simple.py::test_get_specific_user_success PASSED         [100%]

============================== warnings summary ===============================
app\main.py:60
  c:\Users\labadmin\Documents\AG-AISOFTDEV\